In [2]:
import pandas as pd
import xml.etree.ElementTree as ET
from io import StringIO

In [10]:
#id = Seriale
#nom = Nome
#long_parcours = TotLungLato (en m)
#dist_parcouru = TotDistPerc (en m)
#temps_total = TotTempPerc (en s)
#s*_topspeed = cInfoRaceSegment/TopSpeed (en Knt)
#s*_topvmg = cInfoRaceSegment/TopVMG (en Knt)
#s*_avgspeed = cInfoRaceSegment/AvgSpeed (en Knt)
#s*_tempsgauche = cInfoRaceSegment/CrtRaceSegSX (en %)
#s*_tempsdroite = cInfoRaceSegment/CrtRaceSegDX (en %)
#s*_tempssursegment = cInfoRaceSegment/TimeSecPercorsi (en s)
#s*_distancesursegment = cInfoRaceSegment/SegDistRealePercorsa (en m)
#s*_longueursegment = cInfoRaceSegment/LungLato (en m)
#s*_rankentersegment = cInfoRaceSegment/SegEnteredRank

# M7, M8, M9, M10, M11, M12,  Départ – 1 – 4 – 1 – 2 – 3 - Arrivée
# M13  Départ – 1 – 2 – 3 – 2 – 3 - Arrivée
#https://app.metasail.it/(S(usryiluj43x0xh10ztf4whfp))/MetaSailWS.asmx/getStatistiche?idgara=xxx

manche = 'M9'

# remove namespace
src = open('./' + manche + '.xml', 'r').read()
tree = ET.iterparse(StringIO(src))
for _, el in tree:
    prefix, has_namespace, postfix = el.tag.partition('}')
    if has_namespace:
        el.tag = postfix

root = tree.root

df_cols = ['id', 'nom', 'long_parcours', 'dist_parcouru', 'temps_total']
def add_row(colname, value, row):
    row[colname] = value
    if colname not in df_cols:
        df_cols.append(colname)

rows = []
row_index = 0
for node in root:
    row = {
        'id': node.find('Seriale').text if node is not None else None,
        'nom': node.find('Nome').text if node is not None else None,
        'long_parcours': int(node.find('TotLungLato').text) if node is not None else None,
        'dist_parcouru': int(node.find('TotDistPerc').text) if node is not None else None,
        'temps_total': int(node.find('TotTempPerc').text) if node is not None else None,
    }
    curr_segment = 1
    for segment in node.find('lstSegments').iter('cInfoRaceSegment'):
        add_row('s' + str(curr_segment) + '_topspeed', float(segment.find('TopSpeed').text), row)       
        add_row('s' + str(curr_segment) + '_topvmg', float(segment.find('TopVMG').text), row)
        add_row('s' + str(curr_segment) + '_avgspeed', float(segment.find('AvgSpeed').text), row)
        add_row('s' + str(curr_segment) + '_tempsgauche', int(segment.find('CrtRaceSegSX').text), row)
        add_row('s' + str(curr_segment) + '_tempsdroite', int(segment.find('CrtRaceSegDX').text), row)
        add_row('s' + str(curr_segment) + '_tempssursegment', int(segment.find('TimeSecPercorsi').text), row)
        add_row('s' + str(curr_segment) + '_distancesursegment', int(segment.find('SegDistRealePercorsa').text), row)
        add_row('s' + str(curr_segment) + '_longueursegment', int(segment.find('LungLato').text), row)
        add_row('s' + str(curr_segment) + '_rankentersegment', int(segment.find('SegEnteredRank').text), row)
        curr_segment += 1
    rows.append(row)
    
df = pd.DataFrame(rows, columns=df_cols)
df.set_index('id', inplace=True)
print(df.loc[[0]])
df.shape

KeyError: "None of [Int64Index([0], dtype='int64', name='id')] are in the [index]"

In [22]:
#Noel A0708
#Briez A0183
#Ganivez A0535
#Swinyard A0354
#vonck A0309

selection = df[df.index.isin(['A0535', 'A0708', 'A0183', 'A0354', 'A0309'])]

stat_selection = '_avgspeed'#'_distancesursegment'
stat_compare = '_avgspeed'#'_longueursegment'
selection.index
#selection.head()

Index(['A0183', 'A0535', 'A0708', 'A0354', 'A0309'], dtype='object', name='id')

In [23]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

def add_graph(index, fig, df, row, col):
    mean = np.nanmean(df['s' + index + stat_compare])
    text_index = index if index == '6' else int(index) + 1
    t = go.Bar(
        x=selection.nom, 
        y=selection['s' + index + stat_selection],
        name='v moy : ' + str(round(mean, 2)) + 'knt',
        text=selection['s' + str(text_index) + '_rankentersegment']
    )
    
    fig.append_trace(t, row, col)
    fig.add_hline(y=mean, line_color='red', row=row, col=col)

fig_rows = 3
fig_cols = 2

fig = make_subplots(rows=fig_rows, cols=fig_cols, start_cell='top-left', subplot_titles=("Départ -> 1 (près)", "Leg 2", "Leg 3", "Leg 4", "Leg 5", "3 -> arrivée (travers)"))
index = 1
for row in range(fig_rows):
    for col in range(fig_cols):
        add_graph(str(index), fig, df, row + 1, col + 1)
        index += 1


fig.update_layout(title_text= manche + ' :: Vitesse moyenne / flotte par segment', height=900)
fig.update_yaxes(type='log')
fig.show()

In [43]:
content = """<!--<main id="evento-single-container-main-ajax" class="single-content">-->\n    <div class="container-fluid full-width">\n        <!-- single content 428 -->\n        <!-- template-parts/event -->\n<div class="row">\n    <div class="col-md-12">\n        <div class="single-event-header">\n            <h5 class="d-flex align-items-center">Championnats de France Minimes 2023</h5>\n            <p>July 11th - 14th 2023</p>\n        </div>\n    </div>\n</div>\n<div class="row">\n    <div class="col-sm-4">\n        <div class="single-event-logo" style="text-align: center;">\n            <img src="https://app.metasail.it/images/eventi/428.jpg"\n                 alt="Championnats de France Minimes 2023"/>\n        </div>\n\t\t\t\t\n\t\t            <div class="single-event-classi">\n                <div>\n                    <dt>Category</dt>\n\t\t\t\t\t                    <dl>Optimist, Optimist Filles, Optimist Garcons, Open Skiff Garcons, Techno Filles, Techno Garcons, Open Skiff Filles, Feva, Tika</dl>\n                </div>\n            </div>\n\t\t\t\t            <div class="single-event-site">\n                <a href="https://evenements.ffvoile.fr/championnats-de-france-jeunes/cfm-infos-sur-lepreuve/"\n                   target="_blank">Go to the official website</a>\n            </div>\n\t\t    </div>\n    <div class="col-sm-8">\n\t\t\n            <div class="single-block-list">\n\t\t\t\t                    <h4>13/07/2023</h4>\n                    <ul>\n\t\t\t\t\t\t                           \n                            <li>\n                                <div class="event-name" data-order="1_open-skiff-filles_09996">\n                                    <p>\n                                       <a href="https://app.metasail.it/ViewRecordedRace2018.aspx?idgara=20453&token=Y1XU" target="_blank">\n                                            <strong>Open Skiff Filles R 13</strong>\n                                             <span class="event-toolbar">\n                                                                                       \n                                                                                    </span>\n                                        </a>\n                                    </p>\n\t\t\t\t\t\t\t\t\t                                </div>\n\n\n                            </li>\n\t\t\t\t\t\t                           \n                            <li>\n                                <div class="event-name" data-order="1_open-skiff-garcons_09996">\n                                    <p>\n                                       <a href="https://app.metasail.it/ViewRecordedRace2018.aspx?idgara=20452&token=Y1XU" target="_blank">\n                                            <strong>Open Skiff Garcons R 13</strong>\n                                             <span class="event-toolbar">\n                                                                                       \n                                                                                    </span>\n                                        </a>\n                                    </p>\n\t\t\t\t\t\t\t\t\t                                </div>\n\n\n                            </li>\n\t\t\t\t\t\t                           \n                            <li>\n                                <div class="event-name" data-order="1_optimist-filles_09997">\n                                    <p>\n                                       <a href="https://app.metasail.it/ViewRecordedRace2018.aspx?idgara=20454&token=Y1XU" target="_blank">\n                                            <strong>Optimist Filles R 12</strong>\n                                             <span class="event-toolbar">\n                                                                                       \n                                                                                    </span>\n                                        </a>\n                                    </p>\n\t\t\t\t\t\t\t\t\t                                </div>\n\n\n                            </li>\n\t\t\t\t\t\t                           \n                            <li>\n                                <div class="event-name" data-order="1_optimist-filles_09999">\n                                    <p>\n                                       <a href="https://app.metasail.it/ViewRecordedRace2018.aspx?idgara=20447&token=Y1XU" target="_blank">\n                                            <strong>Optimist Filles general recall R 12</strong>\n                                             <span class="event-toolbar">\n                                                                                       \n                                                                                    </span>\n                                        </a>\n                                    </p>\n\t\t\t\t\t\t\t\t\t                                </div>\n\n\n                            </li>\n\t\t\t\t\t\t                           \n                            <li>\n                                <div class="event-name" data-order="1_optimist-garcons_09997">\n                                    <p>\n                                       <a href="https://app.metasail.it/ViewRecordedRace2018.aspx?idgara=20446&token=Y1XU" target="_blank">\n                                            <strong>Optimist Garcons general recall R 12</strong>\n                                             <span class="event-toolbar">\n                                                                                       \n                                                                                    </span>\n                                        </a>\n                                    </p>\n\t\t\t\t\t\t\t\t\t                                </div>\n\n\n                            </li>\n\t\t\t\t\t\t                           \n                            <li>\n                                <div class="event-name" data-order="1_optimist-garcons_09999">\n                                    <p>\n                                       <a href="https://app.metasail.it/ViewRecordedRace2018.aspx?idgara=20459&token=Y1XU" target="_blank">\n                                            <strong>Optimist Garcons abandoned R 12</strong>\n                                             <span class="event-toolbar">\n                                                                                       \n                                                                                    </span>\n                                        </a>\n                                    </p>\n\t\t\t\t\t\t\t\t\t                                </div>\n\n\n                            </li>\n\t\t\t\t\t\t                           \n                            <li>\n                                <div class="event-name" data-order="1_techno-filles_09997">\n                                    <p>\n                                       <a href="https://app.metasail.it/ViewRecordedRace2018.aspx?idgara=20457&token=Y1XU" target="_blank">\n                                            <strong>Techno Filles R 12</strong>\n                                             <span class="event-toolbar">\n                                                                                       \n                                                                                    </span>\n                                        </a>\n                                    </p>\n\t\t\t\t\t\t\t\t\t                                </div>\n\n\n                            </li>\n\t\t\t\t\t\t                           \n                            <li>\n                                <div class="event-name" data-order="1_techno-garcons_09999">\n                                    <p>\n                                       <a href="https://app.metasail.it/ViewRecordedRace2018.aspx?idgara=20456&token=Y1XU" target="_blank">\n                                            <strong>Techno Garcons R 12</strong>\n                                             <span class="event-toolbar">\n                                                                                       \n                                                                                    </span>\n                                        </a>\n                                    </p>\n\t\t\t\t\t\t\t\t\t                                </div>\n\n\n                            </li>\n\t\t\t\t\t\t                    </ul>\n\t\t\t\t                    <h4>12/07/2023</h4>\n                    <ul>\n\t\t\t\t\t\t                           \n                            <li>\n                                <div class="event-name" data-order="1_feva_09988">\n                                    <p>\n                                       <a href="https://app.metasail.it/ViewRecordedRace2018.aspx?idgara=20408&token=Y1XU" target="_blank">\n                                            <strong>Feva R 11</strong>\n                                             <span class="event-toolbar">\n                                                                                       \n                                                                                    </span>\n                                        </a>\n                                    </p>\n\t\t\t\t\t\t\t\t\t                                </div>\n\n\n                            </li>\n\t\t\t\t\t\t                           \n                            <li>\n                                <div class="event-name" data-order="1_feva_09999">\n                                    <p>\n                                       <a href="https://app.metasail.it/ViewRecordedRace2018.aspx?idgara=20399&token=Y1XU" target="_blank">\n                                            <strong>Feva R 10</strong>\n                                             <span class="event-toolbar">\n                                                                                       \n                                                                                    </span>\n                                        </a>\n                                    </p>\n\t\t\t\t\t\t\t\t\t                                </div>\n\n\n                            </li>\n\t\t\t\t\t\t                           \n                            <li>\n                                <div class="event-name" data-order="1_open-skiff-filles_09987">\n                                    <p>\n                                       <a href="https://app.metasail.it/ViewRecordedRace2018.aspx?idgara=20416&token=Y1XU" target="_blank">\n                                            <strong>Open Skiff Filles R 12</strong>\n                                             <span class="event-toolbar">\n                                                                                       \n                                                                                    </span>\n                                        </a>\n                                    </p>\n\t\t\t\t\t\t\t\t\t                                </div>\n\n\n                            </li>\n\t\t\t\t\t\t                           \n                            <li>\n                                <div class="event-name" data-order="1_open-skiff-filles_09988">\n                                    <p>\n                                       <a href="https://app.metasail.it/ViewRecordedRace2018.aspx?idgara=20407&token=Y1XU" target="_blank">\n                                            <strong>Open Skiff Filles R 11</strong>\n                                             <span class="event-toolbar">\n                                                                                       \n                                                                                    </span>\n                                        </a>\n                                    </p>\n\t\t\t\t\t\t\t\t\t                                </div>\n\n\n                            </li>\n\t\t\t\t\t\t                           \n                            <li>\n                                <div class="event-name" data-order="1_open-skiff-filles_09999">\n                                    <p>\n                                       <a href="https://app.metasail.it/ViewRecordedRace2018.aspx?idgara=20393&token=Y1XU" target="_blank">\n                                            <strong>Open Skiff Filles R 10</strong>\n                                             <span class="event-toolbar">\n                                                                                       \n                                                                                    </span>\n                                        </a>\n                                    </p>\n\t\t\t\t\t\t\t\t\t                                </div>\n\n\n                            </li>\n\t\t\t\t\t\t                           \n                            <li>\n                                <div class="event-name" data-order="1_open-skiff-garcons_09987">\n                                    <p>\n                                       <a href="https://app.metasail.it/ViewRecordedRace2018.aspx?idgara=20418&token=Y1XU" target="_blank">\n                                            <strong>Open Skiff Garcons R 12</strong>\n                                             <span class="event-toolbar">\n                                                                                       \n                                                                                    </span>\n                                        </a>\n                                    </p>\n\t\t\t\t\t\t\t\t\t                                </div>\n\n\n                            </li>\n\t\t\t\t\t\t                           \n                            <li>\n                                <div class="event-name" data-order="1_open-skiff-garcons_09988">\n                                    <p>\n                                       <a href="https://app.metasail.it/ViewRecordedRace2018.aspx?idgara=20409&token=Y1XU" target="_blank">\n                                            <strong>Open Skiff Garcons R 11</strong>\n                                             <span class="event-toolbar">\n                                                                                       \n                                                                                    </span>\n                                        </a>\n                                    </p>\n\t\t\t\t\t\t\t\t\t                                </div>\n\n\n                            </li>\n\t\t\t\t\t\t                           \n                            <li>\n                                <div class="event-name" data-order="1_open-skiff-garcons_09999">\n                                    <p>\n                                       <a href="https://app.metasail.it/ViewRecordedRace2018.aspx?idgara=20394&token=Y1XU" target="_blank">\n                                            <strong>Open Skiff Garcons R 10</strong>\n                                             <span class="event-toolbar">\n                                                                                       \n                                                                                    </span>\n                                        </a>\n                                    </p>\n\t\t\t\t\t\t\t\t\t                                </div>\n\n\n                            </li>\n\t\t\t\t\t\t                           \n                            <li>\n                                <div class="event-name" data-order="1_optimist-filles_09988">\n                                    <p>\n                                       <a href="https://app.metasail.it/ViewRecordedRace2018.aspx?idgara=20413&token=Y1XU" target="_blank">\n                                            <strong>Optimist Filles R 11</strong>\n                                             <span class="event-toolbar">\n                                                                                       \n                                                                                    </span>\n                                        </a>\n                                    </p>\n\t\t\t\t\t\t\t\t\t                                </div>\n\n\n                            </li>\n\t\t\t\t\t\t                           \n                            <li>\n                                <div class="event-name" data-order="1_optimist-filles_09999">\n                                    <p>\n                                       <a href="https://app.metasail.it/ViewRecordedRace2018.aspx?idgara=20401&token=Y1XU" target="_blank">\n                                            <strong>Optimist Filles R 10</strong>\n                                             <span class="event-toolbar">\n                                                                                       \n                                                                                    </span>\n                                        </a>\n                                    </p>\n\t\t\t\t\t\t\t\t\t                                </div>\n\n\n                            </li>\n\t\t\t\t\t\t                           \n                            <li>\n                                <div class="event-name" data-order="1_optimist-garcons_09988">\n                                    <p>\n                                       <a href="https://app.metasail.it/ViewRecordedRace2018.aspx?idgara=20411&token=Y1XU" target="_blank">\n                                            <strong>Optimist Garcons R 11</strong>\n                                             <span class="event-toolbar">\n                                                                                       \n                                                                                    </span>\n                                        </a>\n                                    </p>\n\t\t\t\t\t\t\t\t\t                                </div>\n\n\n                            </li>\n\t\t\t\t\t\t                           \n                            <li>\n                                <div class="event-name" data-order="1_optimist-garcons_09989">\n                                    <p>\n                                       <a href="https://app.metasail.it/ViewRecordedRace2018.aspx?idgara=20395&token=Y1XU" target="_blank">\n                                            <strong>Optimist Garcons general recall R 10</strong>\n                                             <span class="event-toolbar">\n                                                                                       \n                                                                                    </span>\n                                        </a>\n                                    </p>\n\t\t\t\t\t\t\t\t\t                                </div>\n\n\n                            </li>\n\t\t\t\t\t\t                           \n                            <li>\n                                <div class="event-name" data-order="1_optimist-garcons_09999">\n                                    <p>\n                                       <a href="https://app.metasail.it/ViewRecordedRace2018.aspx?idgara=20404&token=Y1XU" target="_blank">\n                                            <strong>Optimist Garcons R 10</strong>\n                                             <span class="event-toolbar">\n                                                                                       \n                                                                                    </span>\n                                        </a>\n                                    </p>\n\t\t\t\t\t\t\t\t\t                                </div>\n\n\n                            </li>\n\t\t\t\t\t\t                           \n                            <li>\n                                <div class="event-name" data-order="1_techno-filles_09988">\n                                    <p>\n                                       <a href="https://app.metasail.it/ViewRecordedRace2018.aspx?idgara=20417&token=Y1XU" target="_blank">\n                                            <strong>Techno Filles R 11</strong>\n                                             <span class="event-toolbar">\n                                                                                       \n                                                                                    </span>\n                                        </a>\n                                    </p>\n\t\t\t\t\t\t\t\t\t                                </div>\n\n\n                            </li>\n\t\t\t\t\t\t                           \n                            <li>\n                                <div class="event-name" data-order="1_techno-filles_09989">\n                                    <p>\n                                       <a href="https://app.metasail.it/ViewRecordedRace2018.aspx?idgara=20410&token=Y1XU" target="_blank">\n                                            <strong>Techno Filles R 10</strong>\n                                             <span class="event-toolbar">\n                                                                                       \n                                                                                    </span>\n                                        </a>\n                                    </p>\n\t\t\t\t\t\t\t\t\t                                </div>\n\n\n                            </li>\n\t\t\t\t\t\t                           \n                            <li>\n                                <div class="event-name" data-order="1_techno-filles_09999">\n                                    <p>\n                                       <a href="https://app.metasail.it/ViewRecordedRace2018.aspx?idgara=20405&token=Y1XU" target="_blank">\n                                            <strong>Techno Filles R 9</strong>\n                                             <span class="event-toolbar">\n                                                                                       \n                                                                                    </span>\n                                        </a>\n                                    </p>\n\t\t\t\t\t\t\t\t\t                                </div>\n\n\n                            </li>\n\t\t\t\t\t\t                           \n                            <li>\n                                <div class="event-name" data-order="1_techno-garcons_09988">\n                                    <p>\n                                       <a href="https://app.metasail.it/ViewRecordedRace2018.aspx?idgara=20420&token=Y1XU" target="_blank">\n                                            <strong>Techno Garcons R 11</strong>\n                                             <span class="event-toolbar">\n                                                                                       \n                                                                                    </span>\n                                        </a>\n                                    </p>\n\t\t\t\t\t\t\t\t\t                                </div>\n\n\n                            </li>\n\t\t\t\t\t\t                           \n                            <li>\n                                <div class="event-name" data-order="1_techno-garcons_09989">\n                                    <p>\n                                       <a href="https://app.metasail.it/ViewRecordedRace2018.aspx?idgara=20412&token=Y1XU" target="_blank">\n                                            <strong>Techno Garcons R 10</strong>\n                                             <span class="event-toolbar">\n                                                                                       \n                                                                                    </span>\n                                        </a>\n                                    </p>\n\t\t\t\t\t\t\t\t\t                                </div>\n\n\n                            </li>\n\t\t\t\t\t\t                           \n                            <li>\n                                <div class="event-name" data-order="1_techno-garcons_09990">\n                                    <p>\n                                       <a href="https://app.metasail.it/ViewRecordedRace2018.aspx?idgara=20406&token=Y1XU" target="_blank">\n                                            <strong>Techno Garcons R 9</strong>\n                                             <span class="event-toolbar">\n                                                                                       \n                                                                                    </span>\n                                        </a>\n                                    </p>\n\t\t\t\t\t\t\t\t\t                                </div>\n\n\n                            </li>\n\t\t\t\t\t\t                           \n                            <li>\n                                <div class="event-name" data-order="1_tika_09988">\n                                    <p>\n                                       <a href="https://app.metasail.it/ViewRecordedRace2018.aspx?idgara=20414&token=Y1XU" target="_blank">\n                                            <strong>Tika R 11</strong>\n                                             <span class="event-toolbar">\n                                                                                       \n                                                                                    </span>\n                                        </a>\n                                    </p>\n\t\t\t\t\t\t\t\t\t                                </div>\n\n\n                            </li>\n\t\t\t\t\t\t                           \n                            <li>\n                                <div class="event-name" data-order="1_tika_09999">\n                                    <p>\n                                       <a href="https://app.metasail.it/ViewRecordedRace2018.aspx?idgara=20400&token=Y1XU" target="_blank">\n                                            <strong>Tika R 10</strong>\n                                             <span class="event-toolbar">\n                                                                                       \n                                                                                    </span>\n                                        </a>\n                                    </p>\n\t\t\t\t\t\t\t\t\t                                </div>\n\n\n                            </li>\n\t\t\t\t\t\t                    </ul>\n\t\t\t\t                    <h4>11/07/2023</h4>\n                    <ul>\n\t\t\t\t\t\t                           \n                            <li>\n                                <div class="event-name" data-order="1_feva_09990">\n                                    <p>\n                                       <a href="https://app.metasail.it/ViewRecordedRace2018.aspx?idgara=20362&token=Y1XU" target="_blank">\n                                            <strong>Feva R 9</strong>\n                                             <span class="event-toolbar">\n                                                                                       \n                                                                                    </span>\n                                        </a>\n                                    </p>\n\t\t\t\t\t\t\t\t\t                                </div>\n\n\n                            </li>\n\t\t\t\t\t\t                           \n                            <li>\n                                <div class="event-name" data-order="1_feva_09991">\n                                    <p>\n                                       <a href="https://app.metasail.it/ViewRecordedRace2018.aspx?idgara=20354&token=Y1XU" target="_blank">\n                                            <strong>Feva R 8</strong>\n                                             <span class="event-toolbar">\n                                                                                       \n                                                                                    </span>\n                                        </a>\n                                    </p>\n\t\t\t\t\t\t\t\t\t                                </div>\n\n\n                            </li>\n\t\t\t\t\t\t                           \n                            <li>\n                                <div class="event-name" data-order="1_feva_09992">\n                                    <p>\n                                       <a href="https://app.metasail.it/ViewRecordedRace2018.aspx?idgara=20351&token=Y1XU" target="_blank">\n                                            <strong>Feva R 7</strong>\n                                             <span class="event-toolbar">\n                                                                                       \n                                                                                    </span>\n                                        </a>\n                                    </p>\n\t\t\t\t\t\t\t\t\t                                </div>\n\n\n                            </li>\n\t\t\t\t\t\t                           \n                            <li>\n                                <div class="event-name" data-order="1_open-skiff-filles_09990">\n                                    <p>\n                                       <a href="https://app.metasail.it/ViewRecordedRace2018.aspx?idgara=20359&token=Y1XU" target="_blank">\n                                            <strong>Open Skiff Filles R 9</strong>\n                                             <span class="event-toolbar">\n                                                                                       \n                                                                                    </span>\n                                        </a>\n                                    </p>\n\t\t\t\t\t\t\t\t\t                                </div>\n\n\n                            </li>\n\t\t\t\t\t\t                           \n                            <li>\n                                <div class="event-name" data-order="1_open-skiff-filles_09991">\n                                    <p>\n                                       <a href="https://app.metasail.it/ViewRecordedRace2018.aspx?idgara=20344&token=Y1XU" target="_blank">\n                                            <strong>Open Skiff Filles R 8</strong>\n                                             <span class="event-toolbar">\n                                                                                       \n                                                                                    </span>\n                                        </a>\n                                    </p>\n\t\t\t\t\t\t\t\t\t                                </div>\n\n\n                            </li>\n\t\t\t\t\t\t                           \n                            <li>\n                                <div class="event-name" data-order="1_open-skiff-filles_09992">\n                                    <p>\n                                       <a href="https://app.metasail.it/ViewRecordedRace2018.aspx?idgara=20339&token=Y1XU" target="_blank">\n                                            <strong>Open Skiff Filles R 7</strong>\n                                             <span class="event-toolbar">\n                                                                                       \n                                                                                    </span>\n                                        </a>\n                                    </p>\n\t\t\t\t\t\t\t\t\t                                </div>\n\n\n                            </li>\n\t\t\t\t\t\t                           \n                            <li>\n                                <div class="event-name" data-order="1_open-skiff-garcons_09990">\n                                    <p>\n                                       <a href="https://app.metasail.it/ViewRecordedRace2018.aspx?idgara=20363&token=Y1XU" target="_blank">\n                                            <strong>Open Skiff Garcons R 9</strong>\n                                             <span class="event-toolbar">\n                                                                                       \n                                                                                    </span>\n                                        </a>\n                                    </p>\n\t\t\t\t\t\t\t\t\t                                </div>\n\n\n                            </li>\n\t\t\t\t\t\t                           \n                            <li>\n                                <div class="event-name" data-order="1_open-skiff-garcons_09991">\n                                    <p>\n                                       <a href="https://app.metasail.it/ViewRecordedRace2018.aspx?idgara=20353&token=Y1XU" target="_blank">\n                                            <strong>Open Skiff Garcons R 8</strong>\n                                             <span class="event-toolbar">\n                                                                                       \n                                                                                    </span>\n                                        </a>\n                                    </p>\n\t\t\t\t\t\t\t\t\t                                </div>\n\n\n                            </li>\n\t\t\t\t\t\t                           \n                            <li>\n                                <div class="event-name" data-order="1_open-skiff-garcons_09992">\n                                    <p>\n                                       <a href="https://app.metasail.it/ViewRecordedRace2018.aspx?idgara=20345&token=Y1XU" target="_blank">\n                                            <strong>Open Skiff Garcons R 7</strong>\n                                             <span class="event-toolbar">\n                                                                                       \n                                                                                    </span>\n                                        </a>\n                                    </p>\n\t\t\t\t\t\t\t\t\t                                </div>\n\n\n                            </li>\n\t\t\t\t\t\t                           \n                            <li>\n                                <div class="event-name" data-order="1_optimist-filles_09990">\n                                    <p>\n                                       <a href="https://app.metasail.it/ViewRecordedRace2018.aspx?idgara=20374&token=Y1XU" target="_blank">\n                                            <strong>Optimist Filles R 9</strong>\n                                             <span class="event-toolbar">\n                                                                                       \n                                                                                    </span>\n                                        </a>\n                                    </p>\n\t\t\t\t\t\t\t\t\t                                </div>\n\n\n                            </li>\n\t\t\t\t\t\t                           \n                            <li>\n                                <div class="event-name" data-order="1_optimist-filles_09990">\n                                    <p>\n                                       <a href="https://app.metasail.it/ViewRecordedRace2018.aspx?idgara=20365&token=Y1XU" target="_blank">\n                                            <strong>Optimist Filles abandoned R 9</strong>\n                                             <span class="event-toolbar">\n                                                                                       \n                                                                                    </span>\n                                        </a>\n                                    </p>\n\t\t\t\t\t\t\t\t\t                                </div>\n\n\n                            </li>\n\t\t\t\t\t\t                           \n                            <li>\n                                <div class="event-name" data-order="1_optimist-filles_09991">\n                                    <p>\n                                       <a href="https://app.metasail.it/ViewRecordedRace2018.aspx?idgara=20355&token=Y1XU" target="_blank">\n                                            <strong>Optimist Filles R 8</strong>\n                                             <span class="event-toolbar">\n                                                                                       \n                                                                                    </span>\n                                        </a>\n                                    </p>\n\t\t\t\t\t\t\t\t\t                                </div>\n\n\n                            </li>\n\t\t\t\t\t\t                           \n                            <li>\n                                <div class="event-name" data-order="1_optimist-filles_09992">\n                                    <p>\n                                       <a href="https://app.metasail.it/ViewRecordedRace2018.aspx?idgara=20346&token=Y1XU" target="_blank">\n                                            <strong>Optimist Filles R 7</strong>\n                                             <span class="event-toolbar">\n                                                                                       \n                                                                                    </span>\n                                        </a>\n                                    </p>\n\t\t\t\t\t\t\t\t\t                                </div>\n\n\n                            </li>\n\t\t\t\t\t\t                           \n                            <li>\n                                <div class="event-name" data-order="1_optimist-garcons_09990">\n                                    <p>\n                                       <a href="https://app.metasail.it/ViewRecordedRace2018.aspx?idgara=20368&token=Y1XU" target="_blank">\n                                            <strong>Optimist Garcons general recall R 9</strong>\n                                             <span class="event-toolbar">\n                                                                                       \n                                                                                    </span>\n                                        </a>\n                                    </p>\n\t\t\t\t\t\t\t\t\t                                </div>\n\n\n                            </li>\n\t\t\t\t\t\t                           \n                            <li>\n                                <div class="event-name" data-order="1_optimist-garcons_09990">\n                                    <p>\n                                       <a href="https://app.metasail.it/ViewRecordedRace2018.aspx?idgara=20378&token=Y1XU" target="_blank">\n                                            <strong>Optimist Garcons R 9</strong>\n                                             <span class="event-toolbar">\n                                                                                       \n                                                                                    </span>\n                                        </a>\n                                    </p>\n\t\t\t\t\t\t\t\t\t                                </div>\n\n\n                            </li>\n\t\t\t\t\t\t                           \n                            <li>\n                                <div class="event-name" data-order="1_optimist-garcons_09991">\n                                    <p>\n                                       <a href="https://app.metasail.it/ViewRecordedRace2018.aspx?idgara=20356&token=Y1XU" target="_blank">\n                                            <strong>Optimist Garcons R 8</strong>\n                                             <span class="event-toolbar">\n                                                                                       \n                                                                                    </span>\n                                        </a>\n                                    </p>\n\t\t\t\t\t\t\t\t\t                                </div>\n\n\n                            </li>\n\t\t\t\t\t\t                           \n                            <li>\n                                <div class="event-name" data-order="1_optimist-garcons_09992">\n                                    <p>\n                                       <a href="https://app.metasail.it/ViewRecordedRace2018.aspx?idgara=20347&token=Y1XU" target="_blank">\n                                            <strong>Optimist Garcons R 7</strong>\n                                             <span class="event-toolbar">\n                                                                                       \n                                                                                    </span>\n                                        </a>\n                                    </p>\n\t\t\t\t\t\t\t\t\t                                </div>\n\n\n                            </li>\n\t\t\t\t\t\t                           \n                            <li>\n                                <div class="event-name" data-order="1_techno-filles_09991">\n                                    <p>\n                                       <a href="https://app.metasail.it/ViewRecordedRace2018.aspx?idgara=20371&token=Y1XU" target="_blank">\n                                            <strong>Techno Filles R 8</strong>\n                                             <span class="event-toolbar">\n                                                                                       \n                                                                                    </span>\n                                        </a>\n                                    </p>\n\t\t\t\t\t\t\t\t\t                                </div>\n\n\n                            </li>\n\t\t\t\t\t\t                           \n                            <li>\n                                <div class="event-name" data-order="1_techno-filles_09992">\n                                    <p>\n                                       <a href="https://app.metasail.it/ViewRecordedRace2018.aspx?idgara=20348&token=Y1XU" target="_blank">\n                                            <strong>Techno Filles R 7</strong>\n                                             <span class="event-toolbar">\n                                                                                       \n                                                                                    </span>\n                                        </a>\n                                    </p>\n\t\t\t\t\t\t\t\t\t                                </div>\n\n\n                            </li>\n\t\t\t\t\t\t                           \n                            <li>\n                                <div class="event-name" data-order="1_techno-filles_09993">\n                                    <p>\n                                       <a href="https://app.metasail.it/ViewRecordedRace2018.aspx?idgara=20341&token=Y1XU" target="_blank">\n                                            <strong>Techno Filles R 6</strong>\n                                             <span class="event-toolbar">\n                                                                                       \n                                                                                    </span>\n                                        </a>\n                                    </p>\n\t\t\t\t\t\t\t\t\t                                </div>\n\n\n                            </li>\n\t\t\t\t\t\t                           \n                            <li>\n                                <div class="event-name" data-order="1_techno-garcons_09991">\n                                    <p>\n                                       <a href="https://app.metasail.it/ViewRecordedRace2018.aspx?idgara=20360&token=Y1XU" target="_blank">\n                                            <strong>Techno Garcons R 8</strong>\n                                             <span class="event-toolbar">\n                                                                                       \n                                                                                    </span>\n                                        </a>\n                                    </p>\n\t\t\t\t\t\t\t\t\t                                </div>\n\n\n                            </li>\n\t\t\t\t\t\t                           \n                            <li>\n                                <div class="event-name" data-order="1_techno-garcons_09992">\n                                    <p>\n                                       <a href="https://app.metasail.it/ViewRecordedRace2018.aspx?idgara=20357&token=Y1XU" target="_blank">\n                                            <strong>Techno Garcons R 7</strong>\n                                             <span class="event-toolbar">\n                                                                                       \n                                                                                    </span>\n                                        </a>\n                                    </p>\n\t\t\t\t\t\t\t\t\t                                </div>\n\n\n                            </li>\n\t\t\t\t\t\t                           \n                            <li>\n                                <div class="event-name" data-order="1_techno-garcons_09993">\n                                    <p>\n                                       <a href="https://app.metasail.it/ViewRecordedRace2018.aspx?idgara=20337&token=Y1XU" target="_blank">\n                                            <strong>Techno Garcons R 6</strong>\n                                             <span class="event-toolbar">\n                                                                                       \n                                                                                    </span>\n                                        </a>\n                                    </p>\n\t\t\t\t\t\t\t\t\t                                </div>\n\n\n                            </li>\n\t\t\t\t\t\t                           \n                            <li>\n                                <div class="event-name" data-order="1_tika_09990">\n                                    <p>\n                                       <a href="https://app.metasail.it/ViewRecordedRace2018.aspx?idgara=20364&token=Y1XU" target="_blank">\n                                            <strong>Tika R 9</strong>\n                                             <span class="event-toolbar">\n                                                                                       \n                                                                                    </span>\n                                        </a>\n                                    </p>\n\t\t\t\t\t\t\t\t\t                                </div>\n\n\n                            </li>\n\t\t\t\t\t\t                           \n                            <li>\n                                <div class="event-name" data-order="1_tika_09991">\n                                    <p>\n                                       <a href="https://app.metasail.it/ViewRecordedRace2018.aspx?idgara=20358&token=Y1XU" target="_blank">\n                                            <strong>Tika R 8</strong>\n                                             <span class="event-toolbar">\n                                                                                       \n                                                                                    </span>\n                                        </a>\n                                    </p>\n\t\t\t\t\t\t\t\t\t                                </div>\n\n\n                            </li>\n\t\t\t\t\t\t                           \n                            <li>\n                                <div class="event-name" data-order="1_tika_09992">\n                                    <p>\n                                       <a href="https://app.metasail.it/ViewRecordedRace2018.aspx?idgara=20352&token=Y1XU" target="_blank">\n                                            <strong>Tika R 7</strong>\n                                             <span class="event-toolbar">\n                                                                                       \n                                                                                    </span>\n                                        </a>\n                                    </p>\n\t\t\t\t\t\t\t\t\t                                </div>\n\n\n                            </li>\n\t\t\t\t\t\t                    </ul>\n\t\t\t\t            </div>\n\t\t    </div>\n</div>\n<div class="row">\n    <div class="single-block-list"></div>\n</div>\n<!-- end template-parts/event -->\n        <!-- /single content 428 -->\n    </div>\n<!--</main>-->\n"""

In [67]:
import requests
from bs4 import BeautifulSoup
from w3lib.url import url_query_cleaner

url = 'https://www.metasail.it/wp-content/themes/metasail/ajax/event.php?id=428'
url = url_query_cleaner(url, ('id'), remove=True)
print(url)
# r = requests.get(url)

# c = r.text.removeprefix("b'").removesuffix("'")

# soup = BeautifulSoup(c, 'html.parser')
# for a in soup.find_all('a', href=True):
#    if 'metasail.it' in a['href']:
#        print(a.text, a['href']) 

https://www.metasail.it/wp-content/themes/metasail/ajax/event.php
